In [1]:
!pip install opendatasets


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import opendatasets as od

In [3]:
dataset= 'https://www.kaggle.com/c/fake-news/data?select=train.csv'

In [4]:
od.download(dataset)

Skipping, found downloaded files in ".\fake-news" (use force=True to force download)


In [5]:
import os

In [6]:
import pandas as pd
from nltk.corpus import stopwords

In [7]:
import numpy as np

In [8]:
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
import seaborn as sns
import matplotlib.pyplot as plt

In [10]:
from nltk.stem.porter import PorterStemmer
from string import punctuation

In [11]:
!pip install scikit-learn


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from sklearn.naive_bayes import MultinomialNB

In [13]:
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [15]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [16]:
train_df.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [17]:
test_df.head(5)

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [18]:
train_df.label.value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [19]:
train_df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [20]:
test_df.isna().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [21]:
train_df.fillna("missing",inplace = True)
test_df.fillna("missing",inplace = True)

In [22]:
print("Train Shape: ",train_df.shape)
print("Test Shape: ",test_df.shape)

Train Shape:  (20800, 5)
Test Shape:  (5200, 4)


In [23]:
train_df['text'] = train_df['text'] + " " + train_df['title'] + " " + train_df['author']
del train_df['title']
del train_df['author']
del train_df['id']
test_df['text'] = test_df['text'] + " " + test_df['title'] + " " + test_df['author']
del test_df['title']
del test_df['author']
del test_df['id']

In [24]:
print("Train Shape: ",train_df.shape)
print("Test Shape: ",test_df.shape)

Train Shape:  (20800, 2)
Test Shape:  (5200, 1)


In [25]:
stop = set(stopwords.words('english'))
pnc = list(punctuation)
stop.update(pnc)

In [26]:
from nltk.corpus import stopwords

In [27]:
stop = set(stopwords.words('english'))
pnc = list(punctuation)
stop.update(pnc)

In [28]:
stemmer = PorterStemmer()
def stem_text(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            word = stemmer.stem(i.strip())
            final_text.append(word)
    return " ".join(final_text)

In [29]:
train_df['text'] = (train_df['text'].astype(str)).apply(stem_text)
print('Train done!')
test_df['text'] = (test_df['text'].astype(str)).apply(stem_text)
print('Test done!')
print(train_df.shape)
print(test_df.shape)
train_df

Train done!
Test done!
(20800, 2)
(5200, 1)


,text,label
0,hous dem aide: didn’t even see comey’ letter j...,1
1,ever get feel life circl roundabout rather hea...,0
2,"truth might get fire octob 29, 2016 tension in...",1
3,video 15 civilian kill singl us airstrik ident...,1
4,print iranian woman sentenc six year prison ir...,1
...,...,...
20795,rapper t. i. unload black celebr met donald tr...,0
20796,green bay packer lost washington redskin week ...,0
20797,macy’ today grew union sever great name americ...,0
20798,"nato, russia hold parallel exercis balkan 11/0...",1


In [30]:
X = train_df['text']
y = train_df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [31]:
test_df = test_df['text']

In [32]:
print('X_train shape: ',X_train.shape)
print('y_train shape: ',y_train.shape)
print('X_test shape: ',X_test.shape)
print('y_test shape: ',y_test.shape)
print('test_df shape: ',test_df.shape)

X_train shape:  (16640,)
y_train shape:  (16640,)
X_test shape:  (4160,)
y_test shape:  (4160,)
test_df shape:  (5200,)


In [33]:
cv = CountVectorizer(min_df=0,max_df=1,ngram_range=(1,3))
cv_train =cv.fit_transform(X_train)
cv_test = cv.transform(X_test)
cv_test_df =cv.transform(test_df)
print('Train data shape: ',cv_train.shape)
print('Validation data shape: ',cv_test.shape)
print('Test data shape: ',cv_test_df.shape)

Train data shape:  (16640, 8757575)
Validation data shape:  (4160, 8757575)
Test data shape:  (5200, 8757575)


In [34]:
nb = MultinomialNB()

In [35]:
nb.fit(cv_train, y_train)

MultinomialNB()

In [36]:
pred_nb = nb.predict(cv_test)
score = metrics.accuracy_score(y_test, pred_nb)
print(score)

0.8824519230769231


In [37]:
pred_nbs = nb.predict(cv_test_df)
sub_df = pd.read_csv('submit.csv')
sub_df['label'] = pred_nbs
sub_df.to_csv('sample_sub_nb.csv', index = False)

In [38]:
import joblib
from joblib import dump, load
# Save the model in disk
filename = 'model.sav'
joblib.dump(nb, filename)

# Export vectorizer
filename_vectorizer = 'vectorizer.sav'
joblib.dump(cv, filename_vectorizer)

['vectorizer.sav']

In [39]:
dump(nb, '/nb.joblib')

['/nb.joblib']